# AoC 2024 Day 15
https://adventofcode.com/2024/day/15

In [1]:
import copy

In [2]:
with open('data/day15.txt') as f:
    data = f.read()

MAP, INSTR = data.split('\n\n')
MAP = [list(l) for l in MAP.splitlines()]
INSTR = INSTR.replace('\n', '')

In [3]:
BOX = 'O'
WALL = '#'
ROBOT = '@'
FREE = '.'
LEFT = '<'
RIGHT = '>'
UP = '^'
DOWN = 'v'
DIRS = {LEFT: (0,-1), RIGHT: (0,1), UP: (-1,0), DOWN: (1,0)}

In [4]:
def find_robot(map_data):
    for i, row in enumerate(map_data):
        for j, val in enumerate(row):
            if val == ROBOT:
                return i,j
                
def shift(pos_list, map_data, direction):
    # Set initial robot space to be free
    oi, oj = pos_list[0]
    map_data[oi][oj] = FREE

    # Move the robot
    di, dj = DIRS[direction]
    map_data[oi+di][oj+dj] = ROBOT 

    # If there were boxes, shift all the boxes (effectively adding one box to the end)
    if len(pos_list) > 1:
        fi, fj = pos_list[-1]
        map_data[fi+di][fj+dj] = BOX

    return map_data, (oi+di, oj+dj)
        

def move_robot(pos, map_data, direction):
    curr = pos
    dr, dc = DIRS[direction]
    to_be_shifted = []
    while map_data[curr[0]][curr[1]] != WALL:
        r,c = curr
        if map_data[r][c] == FREE:
            return shift(to_be_shifted, map_data, direction)
        to_be_shifted.append(curr)
        curr = (r+dr, c+dc)
    return map_data, pos

def sum_box_values(map_data):
    total = 0
    for i in range(len(map_data)):
        for j in range(len(map_data[0])):
            if map_data[i][j] == BOX:
                total += 100*i + j
    return total

def simulate():
    map_data = copy.deepcopy(MAP)
    pos = find_robot(map_data)
    for i in INSTR:
        map_data, pos = move_robot(pos, map_data, i)
    return sum_box_values(map_data)

print(simulate())

1429911


## Part 2

In [7]:
DBOX = '[]'
DWALL = '##'
DROBOT = '@.'
DFREE = '..'

def make_double_wide(data):
    return data.replace(BOX, DBOX).replace(WALL, DWALL).replace(FREE, DFREE).replace(ROBOT, DROBOT)

def shift_double(pos_list, map_data, direction):
    map_data_o = copy.deepcopy(map_data)
    dr, dc = DIRS[direction]
    oi, oj = pos_list[0]
    new_pos = [(r+dr,c+dc) for r,c in pos_list]
    new_pos_set = set(new_pos)
    for (r, c),(r2,c2) in zip(pos_list, new_pos):
        map_data[r2][c2] = map_data_o[r][c]
        if (r,c) not in new_pos_set:
            map_data[r][c] = FREE
    return map_data, (oi+dr, oj+dc)

def horizontal_move(pos, map_data, direction):
    dr, dc = DIRS[direction]
    to_be_shifted = []
    curr = pos
    while map_data[curr[0]][curr[1]] != WALL:
        r,c = curr
        if map_data[r][c] == FREE:
            return shift_double(to_be_shifted, map_data, direction)
        to_be_shifted.append(curr)
        curr = (r+dr, c+dc)
    return map_data, pos

def vertical_move(pos, map_data, direction):
    dr, dc = DIRS[direction]
    ir, ic = (pos[0]+dr, pos[1]+dc)
    curr = [(ir,ic)]
    if map_data[ir][ic] == DBOX[0]:
        curr.append((ir,ic+1))
    if map_data[ir][ic] == DBOX[1]:
        curr.append((ir, ic-1))
    to_be_shifted = [pos]
    while all(map_data[r][c] != WALL for r,c in curr):
        if all(map_data[r][c] == FREE for r,c in curr):
            return shift_double(to_be_shifted, map_data, direction)
        layer_shift = []
        for r,c in curr:
            if map_data[r][c] == DBOX[0]:
                layer_shift.extend([(r,c), (r,c+1)])
            elif map_data[r][c] == DBOX[1]:
                layer_shift.extend([(r,c-1), (r,c)])
            elif map_data[r][c] == ROBOT:
                layer_shift.append((r,c))
        to_be_shifted.extend(layer_shift)
        curr = set((r+dr, c+dc) for r,c in layer_shift)
        adjacent = set()
        for r, c in curr:
            if map_data[r][c] == DBOX[0]:
                adjacent.add((r,c+1))
            if map_data[r][c] == DBOX[1]:
                adjacent.add((r, c-1))
        curr |= adjacent
    return map_data, pos

def move_robot_double(pos, map_data, direction):
    if direction in (LEFT, RIGHT):
        return horizontal_move(pos, map_data, direction)
    else:
        return vertical_move(pos, map_data, direction)
    

def simulate_double():
    DMAP = make_double_wide(data.split('\n\n')[0])
    DMAP = [list(l) for l in DMAP.splitlines()]
    map_data = copy.deepcopy(DMAP)
    pos = find_robot(map_data)
    # for r in map_data:
    #     print(r)
    for i in INSTR:
        map_data, pos = move_robot_double(pos, map_data, i)
        # print(i)
        # for r in map_data:
        #     print(r)
    return sum_box_values_double(map_data)

def sum_box_values_double(map_data):
    total = 0
    for i in range(len(map_data)):
        for j in range(len(map_data[0])):
            if map_data[i][j] == DBOX[0]:
                total += 100*i + j
    return total

print(simulate_double())



1453087
